In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import sys, importlib
from torch import nn
from torch.nn import functional as F

sys.path.append('/home/luca_ltpn/DL-Fleuret/Proj_308427_348143_XXXXXX')
sys.path.append('/home/luca_ltpn/DL-Fleuret')

In [2]:
def compute_psnr(x, y, max_range=1.0):
    assert x.shape == y.shape and x.ndim == 4
    return 20 * torch.log10(torch.tensor(max_range)) - 10 * torch.log10(((x-y) ** 2).mean((1,2,3))).mean()

In [3]:
# Import tqdm if installed
try:
    from tqdm import tqdm
except ImportError:
    tqdm = lambda x: x

Model = importlib.import_module(f"Miniproject_1.model").Model
model = Model()
model.load_pretrained_model("/home/luca_ltpn/DL-Fleuret/Proj_308427_348143_XXXXXX/Miniproject_1/bestmodel.pth")

train_path = "/home/luca_ltpn/DL-Fleuret/" + "train_data.pkl"
val_path = "/home/luca_ltpn/DL-Fleuret/" + "val_data.pkl"
train_input0, train_input1 = torch.load(train_path)
val_input, val_target = torch.load(val_path)

train_input0 = (train_input0[:100].float() / 255.0)
train_input1 = (train_input1[:100].float() / 255.0)
val_input = val_input.float() / 255.0
val_target = val_target.float() / 255.0

output_psnr_before=compute_psnr(val_input, val_target)
print(output_psnr_before)

model.train(train_input0, train_input1)

mini_batch_size = 100
model_outputs = []
for b in tqdm(range(0, val_input.size(0), mini_batch_size)):
    output = model.predict(val_input.narrow(0, b, mini_batch_size))
    model_outputs.append(output.cpu())
model_outputs = torch.cat(model_outputs, dim=0)


tensor(20.7239)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10.26it/s]


In [4]:
output_psnr_after = compute_psnr(model_outputs, val_target)
output_psnr_after

tensor(6.3458, grad_fn=<SubBackward0>)